In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
df1 = pd.read_csv('otchet28022018.csv', sep=';', encoding = 'cp1251')

In [3]:
#переименование колонок
old_names = [u'Склад', u'марка', u'Номенклатура', u'Характеристика номенклатуры', u'Конечный остаток'] 
new_names = ['sklad', 'marka', 'nomen', 'harnomen', 'weight']
df1.rename(columns=dict(zip(old_names, new_names)), inplace=True)
print df1.columns
#замена точки на запятую, и привод к числовому формату
df1.weight = df1.weight.str.replace(',','.') 
df1[u'weight'] = df1[u'weight'].astype('float64')
switch = 0

Index([u'sklad', u'nomen', u'harnomen', u'weight'], dtype='object')


In [4]:

if switch == 0: #на случай если запущу этот код 2 раза, в итоге функция не нужна
    df1.insert(loc=len(df1.columns), column='mezhsklad', value=0) #Вставка столбца "межсклад"
    df1.insert(loc=len(df1.columns), column='marka', value=0) #Вставка столбца марка
    df1.insert(loc=len(df1.columns), column='name', value=0) #Вставка столбца Имя продукции
    df1.insert(loc=len(df1.columns), column='peredel', value=0) #Вставка столбца Передел
    switch = 1

# функция проверки на Nan в ячейке
def isNaN(num):
    return num != num

@Обработчик на типовом алгоритме
=====
Если значение в колонке межсклад == 0:
    Если значение в колонке "склад" = u'Прессование ПОСЛЕ АГРЕГАТОВ' присвоить значение в колонке "межсклад" - '1 Прессование после / Обжиг до'
=====

циклы специально завершаются, чтобы следующая обработка запускалась только после заполнения всевозможных ячеек до этого.

In [5]:
i=0
while i < len(df1):
    if df1.loc[i, 'mezhsklad'] == 0:
        if df1.loc[i, 'sklad'] == u'Прессование ПОСЛЕ АГРЕГАТОВ':
            df1.loc[i, 'mezhsklad'] = '1 Прессование после / Обжиг до'
        elif df1.loc[i, 'sklad'] == u'Отделение обжига' or df1.loc[i, 'sklad'] == u'ВПК Отделение обжига':
            df1.loc[i, 'mezhsklad'] = '2 Обжиг в агрегатах'
        elif df1.loc[i, 'sklad'] == u'Отделение пекопропитки':
            df1.loc[i, 'mezhsklad'] = '4 Пропитка в агрегатах'
        elif df1.loc[i, 'sklad'] == u'Отделение графитации' or df1.loc[i, 'sklad'] == u'ВПК Отделение графитации':
            df1.loc[i, 'mezhsklad'] = '7 Графитация в агрегатах'
        elif df1.loc[i, 'sklad'] == u'Графитация ПОСЛЕ АГРЕГАТОВ' or df1.loc[i, 'sklad'] == u'ВПК Графитация ПОСЛЕ АГРЕГАТОВ':
            df1.loc[i, 'mezhsklad'] = '8 Графитация после / Мехобработка до'
    i +=1        

In [6]:
i = 0
while i < len(df1):
    if df1.loc[i, 'mezhsklad'] == 0:
        if df1.loc[i, 'sklad'] == u'Обжиг ДО АГРЕГАТОВ':
            if u'Прессованные' in df1.loc[i,'nomen']:
                df1.loc[i, 'mezhsklad'] = '1 Прессование после / Обжиг до'
    i +=1            

i = 0
while i < len(df1):
    if df1.loc[i, 'mezhsklad'] == 0:
        if df1.loc[i, 'sklad'] == u'Обжиг ДО АГРЕГАТОВ':
            if u'Зеленый' in df1.loc[i,'harnomen']:
                df1.loc[i, 'mezhsklad'] = '1 Прессование после / Обжиг до'
            elif u'Прессованный' in df1.loc[i,'harnomen']:
                df1.loc[i, 'mezhsklad'] = '1 Прессование после / Обжиг до'
            elif u'Пропитанный' in df1.loc[i,'harnomen']:
                df1.loc[i, 'mezhsklad'] = '5 Пропитка после / Обжиг до'
    i +=1  

i = 0
while i < len(df1):
    if df1.loc[i, 'mezhsklad'] == 0:
        if df1.loc[i, 'sklad'] == u'Пропитка ДО АГРЕГАТОВ':
            df1.loc[i, 'mezhsklad'] = '3 Обжиг после / Пропитка до'
        if df1.loc[i, 'sklad'] == u'Пропитка  ПОСЛЕ АГРЕГАТОВ':
            df1.loc[i, 'mezhsklad'] = '5 Пропитка после / Обжиг до'
        if df1.loc[i, 'sklad'] == u'Обжиг ДО АГРЕГАТОВ':
            df1.loc[i, 'mezhsklad'] = '5 Пропитка после / Обжиг до'
        if df1.loc[i, 'sklad'] == u'Графитация ДО АГРЕГАТОВ':
            df1.loc[i, 'mezhsklad'] = '6 Обжиг после / Графитация до'
    i +=1      

In [7]:
i = 0
while i < len(df1):
    if df1.loc[i, 'mezhsklad'] == 0:
        if df1.loc[i, 'sklad'] == u'Обжиг ПОСЛЕ АГРЕГАТОВ' or df1.loc[i, 'sklad'] == u'ВПК Обжиг ПОСЛЕ АГРЕГАТОВ':
            if u'ВПГ' in df1.loc[i,'harnomen'] or u'3ОПГ' in df1.loc[i,'harnomen'] or u'АРВ' in df1.loc[i,'harnomen']:
                if u'Обожженный3' in df1.loc[i,'harnomen'] or u'Обожженный4' in df1.loc[i,'harnomen']:
                    df1.loc[i, 'mezhsklad'] = '6 Обжиг после / Графитация до'
                else:
                    df1.loc[i, 'mezhsklad'] = '3 Обжиг после / Пропитка до'
            if u'Пластина' in df1.loc[i,'nomen'] or u' ГП,' in df1.loc[i,'harnomen']:
                df1.loc[i, 'mezhsklad'] = '9 Обжиг после / Мехобработка до'
            if u'Б-16' in df1.loc[i,'harnomen'] or u'IG' in df1.loc[i,'harnomen'] or u' ИГ' in df1.loc[i,'harnomen']:
                if u'Обожженный1' in df1.loc[i,'harnomen'] or u'Обожженный2' in df1.loc[i,'harnomen']: 
                    df1.loc[i, 'mezhsklad'] = '3 Обжиг после / Пропитка до'
                else:
                    df1.loc[i, 'mezhsklad'] = '6 Обжиг после / Графитация до'
            if u'МГ' in df1.loc[i,'harnomen'] or u'МНГ' in df1.loc[i,'harnomen']:
                if u'Обожженный1' in df1.loc[i,'harnomen'] or u'Обожженный2' in df1.loc[i,'harnomen']:
                    df1.loc[i, 'mezhsklad'] = '3 Обжиг после / Пропитка до'
                else:
                    df1.loc[i, 'mezhsklad'] = '6 Обжиг после / Графитация до'
            if u'ПРОГ' in df1.loc[i,'harnomen']:
                if u'Обожженный1' in df1.loc[i,'harnomen']:
                    df1.loc[i, 'mezhsklad'] = '6 Обжиг после / Графитация до'
            if u'ГМЗ' in df1.loc[i,'harnomen']:
                if u'Обожженный2' in df1.loc[i,'harnomen']:
                    df1.loc[i, 'mezhsklad'] = '6 Обжиг после / Графитация до'
                elif u'Обожженный1' in df1.loc[i,'harnomen'] and u'150, 340' in df1.loc[i,'harnomen']:
                    df1.loc[i, 'mezhsklad'] = '6 Обжиг после / Графитация до'
                else:
                    df1.loc[i, 'mezhsklad'] = '3 Обжиг после / Пропитка до'
    i +=1

In [8]:
i = 0
while i < len(df1):
    if df1.loc[i, 'mezhsklad'] == 0:
        if df1.loc[i, 'sklad'] == u'Обжиг ПОСЛЕ АГРЕГАТОВ' or df1.loc[i, 'sklad'] == u'ВПК Обжиг ПОСЛЕ АГРЕГАТОВ':
            if u'ГНТ' in df1.loc[i,'harnomen'] and u'Обожженный' in df1.loc[i,'harnomen']:
                df1.loc[i, 'mezhsklad'] = '6 Обжиг после / Графитация до'
            if u'Обожженный, БМ, РК, Технический, Норм' in df1.loc[i,'harnomen']:
                df1.loc[i, 'mezhsklad'] = '6 Обжиг после / Графитация до'
            if u'ЭГ' in df1.loc[i,'harnomen'] or u'ГЭ' in df1.loc[i,'harnomen']:
                if u'Обожженный2' in df1.loc[i,'harnomen']:
                    df1.loc[i, 'mezhsklad'] = '6 Обжиг после / Графитация до'
    i +=1                

In [9]:
i = 0
while i < len(df1):
    if df1.loc[i, 'mezhsklad'] == 0:
        if df1.loc[i, 'sklad'] == u'Обжиг ПОСЛЕ АГРЕГАТОВ' or df1.loc[i, 'sklad'] == u'ВПК Обжиг ПОСЛЕ АГРЕГАТОВ':
            if u'ЭГ' in df1.loc[i,'harnomen'] or u'ГЭ' in df1.loc[i,'harnomen']:
                if u'ПФ Ниппель' in df1.loc[i,'nomen']:
                    df1.loc[i, 'mezhsklad'] = '3 Обжиг после / Пропитка до'
                if u'ПФ Электрод' in df1.loc[i,'nomen']:
                    df1.loc[i, 'mezhsklad'] = '6 Обжиг после / Графитация до'
    i +=1

In [10]:
i = 0
while i < len(df1):
    if df1.loc[i, 'mezhsklad'] == 0:
        if df1.loc[i, 'sklad'] == u'Обжиг ПОСЛЕ АГРЕГАТОВ' or df1.loc[i, 'sklad'] == u'ВПК Обжиг ПОСЛЕ АГРЕГАТОВ':
            if u'ЭГ' in df1.loc[i,'harnomen'] or u'ГЭ' in df1.loc[i,'harnomen']:
                if u'300, 300' in df1.loc[i,'harnomen']:
                    df1.loc[i, 'mezhsklad'] = '6 Обжиг после / Графитация до'
                if u'944,' in df1.loc[i,'harnomen']:
                    df1.loc[i, 'mezhsklad'] = '6 Обжиг после / Графитация до'
                if u'710,' in df1.loc[i,'harnomen']:
                    df1.loc[i, 'mezhsklad'] = '6 Обжиг после / Графитация до'
                if u'610,' in df1.loc[i,'harnomen']:
                    df1.loc[i, 'mezhsklad'] = '6 Обжиг после / Графитация до'
                if u'500,' in df1.loc[i,'harnomen']:
                    df1.loc[i, 'mezhsklad'] = '6 Обжиг после / Графитация до'
                if u'200, 1700,' in df1.loc[i,'harnomen']:
                    df1.loc[i, 'mezhsklad'] = '6 Обжиг после / Графитация до'
                if u'250, 250' in df1.loc[i,'harnomen']:
                    df1.loc[i, 'mezhsklad'] = '6 Обжиг после / Графитация до'
    i +=1

In [11]:
i = 0
while i < len(df1):
    if df1.loc[i, 'mezhsklad'] == 0:
        if df1.loc[i, 'sklad'] == u'Обжиг ПОСЛЕ АГРЕГАТОВ' or df1.loc[i, 'sklad'] == u'ВПК Обжиг ПОСЛЕ АГРЕГАТОВ':
            if u'ЭГ' in df1.loc[i,'harnomen'] or u'ГЭ' in df1.loc[i,'harnomen']:
                df1.loc[i, 'mezhsklad'] = '3 Обжиг после / Пропитка до'
    i +=1


In [12]:
z = 0
while z < len(df1):
    if df1.loc[z, 'marka'] == 0:
        if isNaN(df1.loc[z,'harnomen']) == False:
            if u'ЭГ' in df1.loc[z,'harnomen'] or u'ГЭ' in df1.loc[z,'harnomen']:
                df1.loc[z, 'marka'] = 'ЭГ'
            if u'ИГ' in df1.loc[z,'harnomen'] or u'IG' in df1.loc[z,'harnomen'] or u'МГ' in df1.loc[z,'harnomen']:
                df1.loc[z, 'marka'] = 'IG'
            if u'ГМЗ' in df1.loc[z,'harnomen']:
                df1.loc[z, 'marka'] = 'ГМЗ'
            if u'ПРОГ' in df1.loc[z,'harnomen']:
                df1.loc[z, 'marka'] = 'ПРОГ'
            if u'АРВ' in df1.loc[z,'harnomen']:
                df1.loc[z, 'marka'] = 'АРВ'
            if u'ВПГ' in df1.loc[z,'harnomen']:
                df1.loc[z, 'marka'] = 'ВПГ'
            if u'МНГ' in df1.loc[z,'harnomen']:
                df1.loc[z, 'marka'] = 'IG'
            if u'ГП,' in df1.loc[z,'harnomen']:
                df1.loc[z, 'marka'] = 'ГП'
            if u'ГНТ' in df1.loc[z,'harnomen']:
                df1.loc[z, 'marka'] = 'ГНТ'
                
    z +=1
z = 0
while z < len(df1):
    if df1.loc[z, 'marka'] == 0:
        if isNaN(df1.loc[z,'nomen']) == False:
            if u'Пластина' in df1.loc[z,'nomen']:
                df1.loc[z, 'marka'] = 'ПКО'
            if u'ЭГ' in df1.loc[z,'nomen'] or u'ГЭ' in df1.loc[z,'nomen']:
                df1.loc[z, 'marka'] = 'ЭГ'
            if u'ГНТ' in df1.loc[z,'nomen']:
                df1.loc[z, 'marka'] = 'ГНТ'
            if u'ПФ Технический' in df1.loc[z,'nomen']:
                df1.loc[z, 'marka'] = 'ГНТ'
            if u'ПФ Электрод' in df1.loc[z,'nomen']:
                df1.loc[z, 'marka'] = 'ЭГ' 
    z +=1

Далее удаление несоответствующих позиций

In [13]:
i = 0
z = []
while i < len(df1):
    if isNaN(df1.loc[i,'nomen']) == False:
        if u'Хлороподвод графитированный 200*1500, ГЭ' in df1.loc[i,'nomen']:
            z.append(i) #добавление номера строки в массив
        if u'Хладон 12 (дифтордихлорметан)' in df1.loc[i,'nomen']:
            z.append(i)
        if u'Хлор жидкий' in df1.loc[i,'nomen']:
            z.append(i)
        if u'Токоподвод графитированный для ремонта печей' in df1.loc[i,'nomen']:
            z.append(i)
        if u'Температурная труба Ф 200*2100' in df1.loc[i,'nomen']:
            z.append(i)       
    i +=1
df1.drop(z, axis=0, inplace=True) #удаление всех строк, которые были в массиве z

In [14]:
print df1.marka.value_counts() 

ЭГ      89
IG      41
ГМЗ     32
ВПГ     18
ГНТ     13
ПКО     11
ПРОГ     9
АРВ      5
ГП       2
0        1
Name: marka, dtype: int64


In [15]:
#вычленение типа продукции, без указания состояния в 'name' и с указанием состояния в 'peredel'
i=1
p=0
for i in range(0,df1.index[-1]+1):
    try:
        p = df1.harnomen[i].find(',', 0,len(df1.harnomen[i]))
        #print i,'-',df1.harnomen[i][df1.harnomen[i].find(',', 0,len(df1.harnomen[i]))+2:len(df1.harnomen[i])]
        df1.name[i] = df1.harnomen[i][p+2:len(df1.harnomen[i])]
        df1.peredel[i] = df1.harnomen[i][0:p]
        #print df1.name[i]
        print i
        i+=1
    except KeyError: i+=1 
       
    except AttributeError: i+=1 
        


C:\Users\YGalimullin\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\YGalimullin\AppData\Local\Continuum\anaconda2\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\YGalimullin\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name_

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
134
135
136
137
138
139
140
141
142
143
144
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
172
173
174
175
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226


In [16]:
df1[200:]

,sklad,nomen,harnomen,weight,mezhsklad,marka,name,peredel
206,Отделение обжига,ПФ Графит (заготовки),"Пропитанный1, 360, 600, ИГ-172, ПК, Конструкци...",1.3936,2 Обжиг в агрегатах,IG,"360, 600, ИГ-172, ПК, Конструкционный, Норм.",Пропитанный1
207,Отделение обжига,ПФ Графит (заготовки),"Зеленый, 450, 1500, ГМЗ, РК, Конструкционный, ...",31.7760,2 Обжиг в агрегатах,ГМЗ,"450, 1500, ГМЗ, РК, Конструкционный, Норм.",Зеленый
208,Отделение обжига,ПФ Графит (заготовки),"Зеленый, 205, 110, ЭГ-61А, РК, Конструкционный...",0.1560,2 Обжиг в агрегатах,ЭГ,"205, 110, ЭГ-61А, РК, Конструкционный, Норм.",Зеленый
209,Отделение обжига,ПФ Графит (заготовки),"Зеленый, 500, 1500, ГЭ, РК, Конструкционный, Н...",23.7059,2 Обжиг в агрегатах,ЭГ,"500, 1500, ГЭ, РК, Конструкционный, Норм.",Зеленый
210,Отделение обжига,ПФ Графит (заготовки),"Зеленый, 125, 1300, ВПГ, РК, Конструкционный, ...",0.0936,2 Обжиг в агрегатах,ВПГ,"125, 1300, ВПГ, РК, Конструкционный, Норм.",Зеленый
211,Отделение обжига,ПФ Графит (заготовки),"Пропитанный1, 125, 1300, ВПГ, РК, Конструкцион...",50.6548,2 Обжиг в агрегатах,ВПГ,"125, 1300, ВПГ, РК, Конструкционный, Норм.",Пропитанный1
212,Отделение обжига,ПФ Графит (заготовки),"Пропитанный2, 125, 1300, ВПГ, РК, Конструкцион...",29.2656,2 Обжиг в агрегатах,ВПГ,"125, 1300, ВПГ, РК, Конструкционный, Норм.",Пропитанный2
213,Отделение обжига,ПФ Графит (заготовки),"Пропитанный3, 125, 1300, ВПГ, РК, Конструкцион...",0.0326,2 Обжиг в агрегатах,ВПГ,"125, 1300, ВПГ, РК, Конструкционный, Норм.",Пропитанный3
214,Отделение обжига,ПФ Графит (заготовки),"Пропитанный1, 200, 1500, ГМЗ, РК, Конструкцион...",2.4576,2 Обжиг в агрегатах,ГМЗ,"200, 1500, ГМЗ, РК, Конструкционный, Норм.",Пропитанный1
215,Отделение обжига,ПФ Графит (заготовки),"Зеленый, 350, 1500, ГМЗ, РК, Конструкционный, ...",17.1450,2 Обжиг в агрегатах,ГМЗ,"350, 1500, ГМЗ, РК, Конструкционный, Норм.",Зеленый


In [158]:
# Группировка значений колонки "вес" по столбцам межсклад, затем марка - суммирование
df2 = df1.groupby(['mezhsklad','marka'])['weight'].sum() 
df3 = df1.groupby(['marka','mezhsklad'])['weight'].sum()

#Сводная таблица по значениям "вес", строки - "межсклад", столбцы - "марка", суммирование
cikl = pd.DataFrame(pd.pivot_table(df1,'weight','mezhsklad','marka',aggfunc=np.sum))
cikl

marka,0,IG,АРВ,ВПГ,ГМЗ,ГНТ,ГП,ПКО,ПРОГ,ЭГ
mezhsklad,,,,,,,,,,
1 Прессование после / Обжиг до,NaN,4.4406,NaN,NaN,36.2263,626.2542,NaN,22.1940,NaN,591.5835
2 Обжиг в агрегатах,NaN,59.8595,NaN,80.0466,136.4649,538.1350,0.272,24.7649,NaN,697.6068
3 Обжиг после / Пропитка до,NaN,59.9433,NaN,46.9581,19.8535,NaN,NaN,NaN,NaN,328.9409
4 Пропитка в агрегатах,NaN,NaN,NaN,NaN,9.3135,NaN,NaN,NaN,NaN,74.1395
5 Пропитка после / Обжиг до,NaN,10.7097,NaN,55.2342,11.2320,NaN,NaN,1.3178,NaN,77.8028
6 Обжиг после / Графитация до,0.8,1.1984,1.9744,86.3361,42.9643,184.1364,NaN,NaN,21.5366,394.7160
7 Графитация в агрегатах,NaN,NaN,NaN,45.6435,NaN,71.5920,NaN,NaN,NaN,170.6302
8 Графитация после / Мехобработка до,NaN,1.4949,26.8722,42.8840,46.9899,179.7180,NaN,NaN,49.4133,129.9740
9 Обжиг после / Мехобработка до,NaN,NaN,NaN,NaN,NaN,NaN,0.111,50.8797,NaN,NaN


In [159]:
df2 # Как должна выглядеть таблица

mezhsklad                             marka
1 Прессование после / Обжиг до        IG         4.4406
                                      ГМЗ       36.2263
                                      ГНТ      626.2542
                                      ПКО       22.1940
                                      ЭГ       591.5835
2 Обжиг в агрегатах                   IG        59.8595
                                      ВПГ       80.0466
                                      ГМЗ      136.4649
                                      ГНТ      538.1350
                                      ГП         0.2720
                                      ПКО       24.7649
                                      ЭГ       697.6068
3 Обжиг после / Пропитка до           IG        59.9433
                                      ВПГ       46.9581
                                      ГМЗ       19.8535
                                      ЭГ       328.9409
4 Пропитка в агрегатах                ГМЗ        9.3135
    

In [160]:
df3

marka  mezhsklad                           
0      6 Обжиг после / Графитация до             0.8000
IG     1 Прессование после / Обжиг до            4.4406
       2 Обжиг в агрегатах                      59.8595
       3 Обжиг после / Пропитка до              59.9433
       5 Пропитка после / Обжиг до              10.7097
       6 Обжиг после / Графитация до             1.1984
       8 Графитация после / Мехобработка до      1.4949
АРВ    6 Обжиг после / Графитация до             1.9744
       8 Графитация после / Мехобработка до     26.8722
ВПГ    2 Обжиг в агрегатах                      80.0466
       3 Обжиг после / Пропитка до              46.9581
       5 Пропитка после / Обжиг до              55.2342
       6 Обжиг после / Графитация до            86.3361
       7 Графитация в агрегатах                 45.6435
       8 Графитация после / Мехобработка до     42.8840
ГМЗ    1 Прессование после / Обжиг до           36.2263
       2 Обжиг в агрегатах                     136.4649
    

In [161]:
#Подготовка к сохранению файла
df1[u'weight'] = df1[u'weight'].astype('str') #приведение float к строковому формату
df1.weight = df1.weight.str.replace('.',',') #замена точки на запятую

df1.to_csv('otchet28022018-1.csv', sep=';', encoding='cp1251', index=False) #Сохранение в CSV
cikl.to_csv('otchet28022018-svod.csv', sep=';', encoding='cp1251', index=True, header=True) #Сохранение в CSV
df2.to_csv('otchet28022018-svod2.csv', sep=';', encoding='cp1251', index=True, header=True) #Сохранение в CSV
df3.to_csv('otchet28022018-svod3.csv', sep=';', encoding='cp1251', index=True, header=True) #Сохранение в CSV